In [11]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cmocean as cmo
import os,sys,fnmatch,time
import numpy as np
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
from matplotlib.ticker import MultipleLocator
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import animation
from salishsea_tools.nc_tools import scDataset
from matplotlib import ticker
from matplotlib import colors
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap
import math
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as patches

import sys
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/general_circulation/')
import general_functions
import quicklook

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

# Functions

In [12]:
def plot_streamlines_hourly(x_slice, y_slice, u_nstg, v_nstg, speeds, umask, vmask, depthu, actual_hours, day, kind, case):

    #
    umask_slice = umask[:-1, :-1]
    vmask_slice = vmask[:-1, :-1]

    vmin, vmax = 0, 0.7
    cmap = cmo.cm.matter
    cmap.set_bad('silver')
    cmap_mask = LinearSegmentedColormap.from_list('mycmap', ['silver', 'white'])
    norm = mpl.colors.Normalize(vmin=vmin,vmax=vmax)

    fig, axes = plt.subplots(4, 3, figsize = (20, 28)) #
    for ax, n in zip(axes.flatten(), np.arange(12)): #
        plot_speeds = np.ma.array(speeds[n, ...], mask=1 - umask_slice) #
        plot_u_nstg = np.ma.array(u_nstg[n, ...], mask=1 - umask_slice) #
        plot_v_nstg = np.ma.array(v_nstg[n, ...], mask=1 - vmask_slice) #
        strm = ax.streamplot(x_slice, y_slice, plot_u_nstg, plot_v_nstg, color=plot_speeds,
                             cmap=cmap, norm=norm, linewidth=2, density=6, zorder=2) 
        ax.pcolormesh(umask_slice, cmap=cmap_mask, zorder=1)
        plt.setp(ax.get_xticklabels(), visible=False)
        plt.setp(ax.get_yticklabels(), visible=False)
        ax.tick_params(axis='both', which='both', length=0)
        ax.set_aspect(aspect='equal')
        ax.set_xlim([0, x_slice[-1]])
        ax.set_ylim([0, y_slice[-1]])
        ax.set_title('Hour ' + str(actual_hours[n]), fontsize=20) #
        cbar = fig.colorbar(strm.lines, ax=ax, fraction=0.05, orientation='horizontal', pad=0.009)
        cbar.ax.tick_params(labelsize=13)
        tick_locator = ticker.MaxNLocator(nbins=5)
        cbar.locator = tick_locator
        cbar.update_ticks()

    fig.tight_layout(w_pad=1.2, h_pad=0.01)
    fig.suptitle('Day ' + str(day) + ' - ' + kind + ' - ' + case +': Streamlines with speeds [m/s] at depth = '+str(int(depthu[dep_ind]))+' m', fontsize=25)
    plt.subplots_adjust(top=0.96)
    
    return fig

In [13]:
def execute_functions(dirname, fname, dep_ind, day):
    
    time_s = (day-1)*24
    hour_inds = np.arange(1, 24)[::2]
    inds = np.arange(len(hour_inds))
    actual_hours = hour_inds + time_s

    print('day', day)
    print('time_s', time_s)
    print('hour_inds', hour_inds)
    print('inds', inds)
    print('actual_hours', actual_hours)
    
    arrow = 1

    vozocrtx, vomecrty, vozocrtx_avg, vomecrty_avg, umask, vmask, e1u, e2u, depthu, depthv, sozotaux = quicklook.get_uv_at_depth_day(dirname, fname, dep_ind, day)
    
    x_slice, y_slice, u_nstg, v_nstg, speeds = quicklook.get_speeds(vozocrtx, vomecrty, arrow)
    
    speeds_hourly = np.full([len(hour_inds), speeds.shape[-2], speeds.shape[-1]], np.nan)
    u_nstg_hourly = np.full_like(speeds_hourly, np.nan)
    v_nstg_hourly = np.full_like(speeds_hourly, np.nan)
    
        
    for i, hour_ind in zip(inds, hour_inds):
        speeds_hourly[i, :, :] = speeds[hour_ind, :, :]
        u_nstg_hourly[i, :, :] = u_nstg[hour_ind, :, :]
        v_nstg_hourly[i, :, :] = v_nstg[hour_ind, :, :]
        
    fig = plot_streamlines_hourly(x_slice, y_slice, u_nstg, v_nstg, speeds, umask, vmask, depthu, actual_hours, day, kind, case)
    
    fig.savefig('../writing_images/streamlines_'+kind+'_'+case+'_'+str(dep_ind)+'_day'+str(day)+'.png', dpi=100, bbox_inches='tight')
    plt.close(fig) 
    
    return

# Constants

In [14]:
fname = "1_MCKNZ_1h_20170101_201701*"
dep_inds = [1, 9, 16]

# Day 1

In [15]:
day = 1

In [16]:
kind = 'ideal'
case = 'base' 

for dep_ind, n in zip(dep_inds, np.arange(len(dep_inds))):
    dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'
    execute_functions(dirname, fname, dep_ind, day)

day 1
time_s 0
hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [ 1  3  5  7  9 11 13 15 17 19 21 23]
day 1
time_s 0
hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [ 1  3  5  7  9 11 13 15 17 19 21 23]
day 1
time_s 0
hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [ 1  3  5  7  9 11 13 15 17 19 21 23]


In [17]:
kind = 'real'
case = 'base' 

for dep_ind, n in zip(dep_inds, np.arange(len(dep_inds))):
    dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'
    execute_functions(dirname, fname, dep_ind, day)

day 1
time_s 0
hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [ 1  3  5  7  9 11 13 15 17 19 21 23]
day 1
time_s 0
hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [ 1  3  5  7  9 11 13 15 17 19 21 23]
day 1
time_s 0
hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [ 1  3  5  7  9 11 13 15 17 19 21 23]


# Day 2

In [18]:
day = 2

In [19]:
kind = 'ideal'
case = 'base' 

for dep_ind, n in zip(dep_inds, np.arange(len(dep_inds))):
    dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'
    execute_functions(dirname, fname, dep_ind, day)

day 2
time_s 24
hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [25 27 29 31 33 35 37 39 41 43 45 47]
day 2
time_s 24
hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [25 27 29 31 33 35 37 39 41 43 45 47]
day 2
time_s 24
hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [25 27 29 31 33 35 37 39 41 43 45 47]


In [20]:
kind = 'real'
case = 'base' 

for dep_ind, n in zip(dep_inds, np.arange(len(dep_inds))):
    dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'
    execute_functions(dirname, fname, dep_ind, day)

day 2
time_s 24
hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [25 27 29 31 33 35 37 39 41 43 45 47]
day 2
time_s 24
hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [25 27 29 31 33 35 37 39 41 43 45 47]
day 2
time_s 24
hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [25 27 29 31 33 35 37 39 41 43 45 47]
